<a href="https://colab.research.google.com/github/abhi-11nav/Rice_classification/blob/main/Rice_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd 
import numpy as np

In [26]:
# Importing libraries for buliding the model 

import tensorflow 
from tensorflow import keras 

from keras.layers import Dense, Flatten, Dropout 
from keras.models import Model 

from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.vgg19 import VGG19

from glob import glob

# Visualization libraries 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

In [27]:
# Checking the version of GPU 

!nvidia-smi

Fri Aug 19 19:56:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |   9214MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Data Processing

In [28]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
# Importing splitfolder library for data seperation 

import splitfolders

In [30]:
# Split folders allows us to seperate training and testing data

"""
input_path = '/content/drive/MyDrive/Rice_Image_Dataset/Rice varieties'

output_path = '/content/drive/MyDrive/Rice_Image_Dataset/Rice dataset'

splitfolders.ratio(input_path, output=output_path,seed=42, ratio=(.78,.22))
"""

"\ninput_path = '/content/drive/MyDrive/Rice_Image_Dataset/Rice varieties'\n\noutput_path = '/content/drive/MyDrive/Rice_Image_Dataset/Rice dataset'\n\nsplitfolders.ratio(input_path, output=output_path,seed=42, ratio=(.78,.22))\n"

In [31]:
# Providing the train and validation paths 

train_path = "/content/drive/MyDrive/Rice_Image_Dataset/Rice dataset/train"

validation_path = "/content/drive/MyDrive/Rice_Image_Dataset/Rice dataset/val"

In [32]:
train_datagen = ImageDataGenerator(vertical_flip = True, horizontal_flip=True, zoom_range=0.2, rescale=1./255, shear_range=0.2)

validation_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
training_data = train_datagen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', batch_size =16)
validation_data = train_datagen.flow_from_directory(validation_path, target_size=(224,224),class_mode='categorical',batch_size =16)

Found 56382 images belonging to 5 classes.
Found 15906 images belonging to 5 classes.


In [34]:
training_data.class_indices

{'Arborio': 0, 'Basmati': 1, 'Ipsala': 2, 'Jasmine': 3, 'Karacadag': 4}

## Building the Model 

In [35]:
# Calling the model function by specifying the input shape and retaining the imagenet weights

model_ = VGG19(include_top=False, weights='imagenet', input_shape = (224,224,3))

In [36]:
# Setting the layers trainable property to False. So we only train the end layers of the model 

for layers in model_.layers:
  layers.trainable = False

In [37]:
# Attaching additional flatten and output layer at the end of the model 

# Flatten layer 1 
flatten_layer = Flatten()(model_.output)

# Dense layer as the output layer with 5 units (output labels) and softmax activation function 
prediction_layer = Dense(5, activation="softmax")(flatten_layer)

In [38]:
# Final model with input an output arguments passed

final_model = Model(inputs=model_.input, outputs=prediction_layer)

In [39]:
# Summary of the model we are going to use on our data 

final_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [40]:
# Compiling the model 

final_model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [41]:
# Checking for GPU version 

!nvidia-smi 

Fri Aug 19 19:56:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |   9214MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
# Fitting the model onto train and validation data

history = final_model.fit(training_data, validation_data = validation_data, epochs = 10)

KeyboardInterrupt: ignored

In [ ]:
final_model.save("model.h5")